In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
from holoviews import opts
import panel as pn
pn.extension()
hv.extension('bokeh')
import geopandas as gpd
from shapely.geometry import Point
import folium
import branca.colormap as cm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from IPython.core.display import HTML


In [ ]:
data_csv_file="data/commute/CSVLog_20241021_060005.CSV"
output_folder_path = 'output/date/analysis_20241021_am/'  # Change this to your folder path
#create folder folder if it does not exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [ ]:
def filter(df,column,min,max): 
    mask = (df[f'{column}'] >= min) & (df[f'{column}'] < max)
    df = df[mask]
    return df

In [ ]:
def create_map(df,output_folder_path,output_file_name):
    df_map=df.copy()
    map_center = [44.980164, -93.185692]
    map = folium.Map(location=map_center, zoom_start=11)
    coordinates = df_map[['lat', 'long', 'MPH']].values.tolist()
    sorted_coordinates = sorted(coordinates, key=lambda x: x[2])
    colormap = cm.LinearColormap(
        colors=['blue', 'green', 'yellow', 'red'],  # Custom color range
        vmin=df_map['MPH'].min(),
        vmax=df_map['MPH'].max(),
        caption='MPH'   
    )

    colormap.add_to(map)



    for i in range(len(df_map) - 1):
        # Coordinates of the current segment
        start = [df_map.iloc[i]['lat'], df_map.iloc[i]['long']]
        end = [df_map.iloc[i + 1]['lat'], df_map.iloc[i + 1]['long']]

        # Value for color mapping (use average or a specific value)
        segment_value = (df_map.iloc[i]['MPH'] + df_map.iloc[i + 1]['MPH']) / 2

        # Create and add a PolyLine for each segment
        folium.PolyLine(
            locations=[start, end],
            color=colormap(segment_value),  # Map value to color
            weight=5,
            opacity=1
            ).add_to(map)

    # Save the map to an HTML file
    map_moving_file_path = os.path.join(output_folder_path, f'{output_file_name}.html')
    map.save(map_moving_file_path)
    return map

In [ ]:
df=pd.read_csv(data_csv_file)
df.columns = df.columns.str.strip()
column_list=df.columns.tolist()

formatted_columns = [f'"{column}",' for column in column_list]
# Join them together with newlines
formatted_string = '\n'.join(formatted_columns)

print(formatted_string)

In [ ]:
current_column_names=[
       'Trip Distance (miles)', 
       'Vehicle speed (MPH)',
       'Relative accelerator pedal position (%)',
       "Relative throttle position (%)",
       'Acceleration (ft/s²)',
       'Acceleration (Avg) (ft/s²)',
       'Engine Power (hp)', 
       'Engine RPM (RPM)',
       'Mass air flow rate (lb/min)',
       'Intake manifold absolute pressure (inHg)',
       'Commanded fuel rail pressure A (inHg)',
       'Fuel rail pressure A (inHg)',
       'Ambient air temperature (°F)',
       'Intake air temperature bank 1  sensor 1 (°F)',
       'Intake air temperature bank 1  sensor 2 (°F)',
       'Instant fuel economy (MPG)',
       'Trip Fuel Economy (MPG)',
       'Barometric pressure (inHg)',
       'Altitude (ft)',
       'Latitude (deg)',
       'Longitude (deg)',
       'Fuel type',
       'Hard Brake Count',
       'Calculated load value (%)',
       'Engine coolant temperature (°F)',
       'Catalyst temperature (Bank 1 Sensor 1) (°F)',
       'Engine oil temperature (°F)',
       'Trip Fuel (gal)',
       'Average Trip CO2 Rate (lb/mile)',
       'Engine Torque (lb•ft)'
       ]
new_column_names = [
       'distTrip (Miles)',
       'MPH',
       'relAccPedPOS (%)',
       'relThrotPOS (%)',
       'accel (ft/s²)',
       'accelAvg (ft/s²)',
       'HP',
       'RPM',
       'massAirFlowRate (lb/min)',
       'inManAbsPress (inHg)',
       'fuelRailPressA CMD (inHg)',
       'fuelRailPressA (inHg)',
       'tempAmbAir (°F)',
       'tempInBnk1Sens1 (°F)',
       'tempInBnk1Sens2 (°F)',
       'mpgInstant',
       'mpgTrip',
       'barPress (inHg)',
       'alt (ft)',
       'lat',
       'long',
       'fuelType',
       'brkHardCnt',
       'loadValCalc (%)',
       'tempCool (°F)',
       'tempCatBnk1Sens1 (°F)',
       'tempOil (°F)',
       'fuelTrip (gal)',
       'avgTripCO2Rate (lb/mile)',
       'Torque (lb•ft)'
       ]

rename_dict = dict(zip(current_column_names, new_column_names))

# Filter the rename_dict to only include columns that exist in the DataFrame
filtered_rename_dict = {k: v for k, v in rename_dict.items() if k in df.columns}

# Rename columns based on the filtered mapping
df = df.rename(columns=filtered_rename_dict)
df.head()

In [ ]:
df = df.rename(columns=rename_dict)
df.head()

In [ ]:
# To avoid math errors in situations where values = 0
df['Boost (psi)'] = df['Boost (psi)'].bfill().infer_objects()
df.loc[df['Boost (psi)'] == 0, 'Boost (psi)'] = 0.01

df.head(2)


In [ ]:
def clean_data(df):
    # Round columns 'Time (sec)', ' Engine RPM (RPM)' and 2 other columns (Number of decimals: 0)
    df = df.round({
                'RPM': 0,
                'Engine Power (hp)': 1,
                'fuelRailPressA CMD (inHg)':0,
                'fuelRailPressA (inHg)':0,
                'Altitude (ft)':0,
                'mpgInstant': 2,
                'MPH': 2,
                'relAccPedPOS (%)': 2,
                'mpgTrip': 2,
                'Boost (psi)':2,
                'inManAbsPress (inHg)': 2,
                'massAirFlowRate (lb/min)': 2,
                'Bearing (deg)': 2,
                'accel (ft/s²)': 3,
                'accelAvg (ft/s²)': 3,
                'Time (sec)': 3,
                'distTrip (Miles)': 3,
                'barPress (inHg)':3,
                'alt (ft)':1,
                'HP':2
                })
    return df

df_clean = clean_data(df.copy())
df_clean.head(2)

In [ ]:
df_clean.describe()

In [ ]:
df_clean.columns

In [ ]:
#create dataframe where vehicle is stationary
df_stationary = df_clean[df_clean['MPH'] == 0]

In [ ]:
# Shift the 'Vehicle speed (MPH)' column
df_clean['next_mph'] = df_clean['MPH'].shift(-1)

# Create the mask
mask = (df_clean['MPH'] > 0) | ((df_clean['MPH'] == 0) & (df_clean['next_mph'] > 0))
mask = mask & df_clean['next_mph'].notna()
# Filter the DataFrame and drop the 'next_mph' column
df_moving = df_clean[mask].drop(columns=['next_mph'])
df_moving = df_clean[mask].drop(columns=['next_mph'])
df_moving.describe()


In [ ]:
if 'alt (ft)' in df_moving.columns:
    df_moving.hvplot(
        kind='scatter',
        x='distTrip (Miles)',
        y='alt (ft)',  # Single column doesn't need to be wrapped in a list
        legend='bottom_right',
        widget_location='bottom',
    )

In [ ]:
df_0_10=filter(df_moving,'MPH',0,10)
df_0_10.describe()


In [ ]:
hvexplorer = df_0_10.hvplot.explorer()
hvexplorer

In [ ]:
df_10_20=filter(df_moving,'MPH',10,20)
df_10_20.describe()

In [ ]:
hvexplorer = df_10_20.hvplot.explorer()
hvexplorer

In [ ]:
df_20_30=filter(df_moving,'MPH',20,30)
df_20_30.describe()


In [ ]:
hvexplorer = df_20_30.hvplot.explorer()
hvexplorer


In [ ]:
df_30_40=filter(df_moving,'MPH',30,40)
df_30_40.describe()

In [ ]:
hvexplorer = df_30_40.hvplot.explorer()
hvexplorer

In [ ]:
df_40_50=filter(df_moving,'MPH',40,50)
df_40_50.describe()

In [ ]:
hvexplorer = df_40_50.hvplot.explorer()
hvexplorer

In [ ]:
df_50_60=filter(df_moving,'MPH',50,60)
df_50_60.describe()

In [ ]:
hvexplorer = df_50_60.hvplot.explorer()
hvexplorer

In [ ]:
df_60_70=filter(df_moving,'MPH',60,70)
df_60_70.describe()

In [ ]:
hvexplorer = df_60_70.hvplot.explorer()
hvexplorer

In [ ]:
df_70_85=filter(df_moving,'MPH',70,85)
df_70_85.describe()

In [ ]:
hvexplorer = df_70_85.hvplot.explorer()
hvexplorer

In [ ]:
df_55_80=filter(df_moving,'MPH',55,80)
df_55_80.describe()

In [ ]:
map_55_80 = create_map(df_55_80,output_folder_path,"map_55_80")
map_55_80

In [ ]:
if 'fuelRailPressA CMD (inHg)' in df_moving.columns:
    df_fuel_rail = df_55_80[['RPM','fuelRailPressA CMD (inHg)', 'fuelRailPressA (inHg)']].copy()
    df_fuel_rail['cmd/act'] = df_fuel_rail['fuelRailPressA CMD (inHg)'] / df_fuel_rail['fuelRailPressA (inHg)'].replace(0, np.nan)
    df_fuel_rail['cmd-act'] = df_fuel_rail['fuelRailPressA CMD (inHg)'] - df_fuel_rail['fuelRailPressA (inHg)'].replace(0, np.nan)
    df_fuel_rail.describe()

In [ ]:
hvexplorer = df_fuel_rail.hvplot.explorer()
hvexplorer

In [ ]:
hvexplorer = df_55_80.hvplot.explorer()
hvexplorer

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import hvplot.pandas

# Sample DataFrame
#data = {
#    'square_footage': [1500, 1600, 1700, 1800, 1900],
#    'num_bedrooms': [3, 3, 4, 4, 5],
#    'age': [10, 15, 20, 25, 30],
#    'price': [300000, 320000, 340000, 360000, 380000]
#}
#df = pd.DataFrame(data)

# Independent variables
X = df_clean[['RPM', 'massAirFlowRate (lb/min)']]
#print(X.shape)
# Dependent variable
y = df_clean['HP']

# Create and train the model
model = LinearRegression()
model.fit(X, y)

# Make predictions
df_clean['predicted_hp'] = model.predict(X)

# Plotting the actual vs predicted prices using hvplot
plot = df_clean.hvplot.scatter(
    x='Time (sec)', 
    y='HP',
    color='blue',
    label='Actual HP'
) * df_clean.hvplot.line(
    x='Time (sec)',
    y='predicted_hp',
    color='red',
    label='Predicted HP'
)

# Set plot size, title, labels, etc.
plot = plot.opts(
    title="Actual vs Predicted HP", 
    xlabel="Square RPM", 
    ylabel="HP",
    width=1200,  # Adjust width as needed
    height=600  # Adjust height as needed
)

plot



In [ ]:

df_predict_hp=df_clean[['RPM', 'massAirFlowRate (lb/min)','HP','predicted_hp']]
df_predict_hp.reset_index()
df_predict_hp.head()

In [ ]:
hvexplorer=df_predict_hp.hvplot.explorer()
hvexplorer